In [10]:
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.serialize import model_to_json, model_from_json
import pandas as pd
import json
from datetime import timedelta
import numpy as np
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s :: %(levelname)s :: %(message)s")
from glob import glob
from tqdm import tqdm
import os
import glob


In [19]:
def load_model_from_json(file_path):
    with open(file_path, "rb") as file:
        prophet_model = model_from_json(file.read())
        
    return prophet_model

In [114]:
df_grouped_by_week = pd.read_parquet('./datasets/master_sales_by_week_curated.parquet')
incremento_precios_familia = pd.read_csv('./archivos_insumo/incremento_precios_familia.csv')
archivo_descuentos = pd.read_csv('./archivos_insumo/archivo_insumo_2023.csv')
archivo_descuentos['ds'] = pd.to_datetime(archivo_descuentos['ds'])

In [122]:
df_grouped_by_week_2024 = df_grouped_by_week[df_grouped_by_week['date'].between('2024-01-01','2024-12-31')]
df_grouped_by_week_2024 = df_grouped_by_week_2024[['date','combination','discount']]
df_grouped_by_week_2024['familia'] = df_grouped_by_week_2024['combination'].apply(lambda x: x[:3])
df_grouped_by_week_2024['ds'] = df_grouped_by_week_2024['date']
del df_grouped_by_week_2024['date']
df_grouped_by_week_2024 = df_grouped_by_week_2024[['ds','familia','discount']]
df_grouped_by_week_2024['discount'] = df_grouped_by_week_2024['discount'].apply(lambda x: np.random.randint(0, 71) )
df_grouped_by_week_2024 = df_grouped_by_week_2024.groupby(['ds','familia'], as_index=False)['discount'].min()
df_grouped_by_week_2024
df_grouped_by_week_2024.to_csv('./archivos_insumo/archivo_insumo_dummy_campañas_2024.csv', index = False)

In [5]:
# Lets use data 2023 as example
df_grouped_by_week_2022 = df_grouped_by_week[df_grouped_by_week['date_week'].between('2022-01-01','2022-12-31')]
df_grouped_by_week_2023 = df_grouped_by_week[df_grouped_by_week['date_week'].between('2023-01-01','2023-12-31')]

In [28]:
# Get all the combinations trained
models_list = [ os.path.basename(i).split('.')[0] for i in glob.glob('./serialized_models/*')]

In [48]:
df_grouped_by_week_2022

,combination,date_week,quantity,discount_for_event,campaigns_name,price_taxes_excluded,campaign,cod_fami
179,201AA3,2022-01-10,3,0.000000,no-discount,49554.628906,0,201
180,201AA3,2022-01-17,6,8.000000,no-discount,73067.234375,0,201
181,201AA3,2022-01-24,6,40.000000,SALE,67184.875000,1,201
182,201AA3,2022-01-31,1,40.000000,SALE,10915.969727,1,201
183,201AA3,2022-02-07,3,13.333333,no-discount,43672.281250,0,201
...,...,...,...,...,...,...,...,...
451766,281310462AH2157,2022-11-07,5,0.000000,no-discount,79789.906250,0,281
451767,281310462AH2157,2022-11-21,2,0.000000,no-discount,31915.960938,0,281
451768,281310462AH2157,2022-11-28,1,10.000000,BLACK WEEK,9574.790039,1,281
451976,281348434AJ2157,2022-06-06,2,0.000000,no-discount,12605.040039,0,281


In [47]:
df_grouped_by_week_2022['cod_fami'] = df_grouped_by_week_2022['combination'].apply(lambda x: x[:3])

In [89]:
model = load_model_from_json('./serialized_models/201AA3.json')
combination = '201AA3'
cod_fami = str(combination[:3])

year = 2023
# Obtener la última fecha histórica
year_to_forecast = 2023

# Crear un DataFrame con las fechas futuras hasta el año especificado, por semana comenzando en lunes
future_dates = pd.date_range(start=f'{year_to_forecast}-01-01', end=f'{year}-12-31', freq='W-MON')

# Crear el DataFrame con las fechas y los regresores
future_df = pd.DataFrame({'ds': future_dates})

# Añadir los regresores al DataFrame futuro
archivo_descuentos = archivo_descuentos[archivo_descuentos['familia'] == int(cod_fami)]
future_df = future_df.merge(archivo_descuentos, how='left', on ='ds')

# Llenar NaNs con 0 si es necesario 
future_df['familia'].fillna(int(cod_fami), inplace=True)
future_df['discount'].fillna(0, inplace=True)

df_2022_precio = df_grouped_by_week_2022[df_grouped_by_week_2022['combination'] == combination][['date_week','combination','price_taxes_excluded']]
incremento_porcentual = incremento_precios_familia[incremento_precios_familia['familia'] == int(cod_fami)]['incremento_porcentual'].iat[0]
df_2022_precio['incremento_porcentual'] = incremento_porcentual / 100
df_2022_precio['price_taxes_excluded'] = (df_2022_precio['price_taxes_excluded'] * df_2022_precio['incremento_porcentual']) + df_2022_precio['price_taxes_excluded'] 

future_df['price_taxes_excluded'] = df_2022_precio['price_taxes_excluded'].mean()

/Users/jdaza/.pyenv/versions/3.11.0/lib/python3.11/site-packages/prophet/serialize.py:172: FutureWarning: The behavior of Timestamp.utcfromtimestamp is deprecated, in a future version will return a timezone-aware Timestamp with UTC timezone. To keep the old behavior, use Timestamp.utcfromtimestamp(ts).tz_localize(None). To get the future behavior, use Timestamp.fromtimestamp(ts, 'UTC')
  setattr(model, attribute, pd.Timestamp.utcfromtimestamp(model_dict[attribute]).tz_localize(None))


In [94]:

# Hacer la predicción
forecast = model.predict(future_df)
forecast = forecast[['ds','yhat']]
forecast['prediccion'] = np.ceil(forecast['yhat'])

In [95]:
forecast

,ds,yhat,prediccion
0,2023-01-02,0.776950,1.0
1,2023-01-09,0.867674,1.0
2,2023-01-16,1.538576,2.0
3,2023-01-23,2.302693,3.0
4,2023-01-30,2.414771,3.0
5,2023-02-06,1.966814,2.0
6,2023-02-13,1.495903,2.0
7,2023-02-20,1.562022,2.0
8,2023-02-27,2.063034,3.0
9,2023-03-06,2.379502,3.0
